In [ ]:
import esm
import torch
import pandas as pd

In [ ]:
model, alphabet = esm.pretrained.esm2_t48_15B_UR50D()
model.eval()

In [ ]:
df = pd.read_csv('data/OsmoticStress_with_binary_positions.csv')

In [ ]:
def generate_embeddings(model, alphabet, sequence):
    # Tokenize and prepare input
    data = [(0, sequence)]
    batch_converter = alphabet.get_batch_converter()
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    # Generate embeddings
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6])  # Extract representations from the last layer
        token_embeddings = results['representations'][6]

    # Average embeddings across all tokens
    return token_embeddings.mean(dim=1).numpy()

# Assuming 'model' and 'alphabet' are already loaded as shown earlier
df['Full_embedding'] = df['full_sequence'][:2].apply(lambda x: generate_embeddings(model, alphabet, x))